In [60]:
from lxml import etree
from shapely.geometry import Polygon
import folium
import json
from branca.element import Template, MacroElement

# List your S1 KML files with corresponding satellite names
kml_files = [
    # ('SatelliteData//S1A_MP_USER_20250717T171636_20250806T194000.kml', 'S1A'),
    ('SatelliteData//S1C_MP_USER_20250718T173834_20250807T194000.kml', 'S1C'),
]

ns = {'kml': 'http://www.opengis.net/kml/2.2'}

def extract_date_folders(document):
    date_folders = []
    for elem in document:
        if etree.QName(elem.tag).localname == 'Folder':
            name_el = elem.find('kml:name', ns)
            name = name_el.text if name_el is not None else ''
            # Check if it has nested Folder (heuristic for date folder)
            if len(elem) == 2 and elem[1].tag.endswith('Folder'):
                date_folders.append((name, elem))
    return date_folders

def extract_polygons_from_folder(folder):
    polygons = []
    metas = []
    for placemark in folder:
        if etree.QName(placemark.tag).localname != 'Placemark':
            continue
        
        name_el = placemark.find('kml:name', ns)
        name = name_el.text if name_el is not None else 'NoName'
        
        timespan = placemark.find('kml:TimeSpan', ns)
        begin = timespan.find('kml:begin', ns).text if timespan is not None else None
        end = timespan.find('kml:end', ns).text if timespan is not None else None
        
        linear_ring = placemark.find('kml:LinearRing', ns)
        if linear_ring is None:
            continue
        
        coords_text = linear_ring.find('kml:coordinates', ns).text.strip()
        coords_list = []
        for c in coords_text.split():
            lon, lat, _ = c.split(',')
            coords_list.append((float(lon), float(lat)))
        
        poly = Polygon(coords_list)
        polygons.append(poly)
        metas.append({'name': name, 'begin': begin, 'end': end})
    return polygons, metas

def parse_kml_file(kml_file, satellite_name):
    with open(kml_file, 'rb') as f:
        tree = etree.parse(f)
    root = tree.getroot()
    document = root.find('kml:Document', ns)
    date_folders = extract_date_folders(document)

    date_polygons = {}
    for date_name, folder in date_folders:
        nested_folder = None
        for child in folder:
            if etree.QName(child.tag).localname == 'Folder':
                nested_folder = child
                break
        if nested_folder is None:
            continue
        
        polygons, metas = extract_polygons_from_folder(nested_folder)
        # Add satellite info to metas
        for meta in metas:
            meta['satellite'] = satellite_name
        
        if date_name in date_polygons:
            date_polygons[date_name]['polygons'].extend(polygons)
            date_polygons[date_name]['metas'].extend(metas)
        else:
            date_polygons[date_name] = {'polygons': polygons, 'metas': metas}
    return date_polygons

# Parse both files and merge their data
combined_date_polygons = {}

for kml_file, sat_name in kml_files:
    print(f"Parsing {kml_file} ...")
    dp = parse_kml_file(kml_file, sat_name)
    for date_name, data in dp.items():
        if date_name in combined_date_polygons:
            combined_date_polygons[date_name]['polygons'].extend(data['polygons'])
            combined_date_polygons[date_name]['metas'].extend(data['metas'])
        else:
            combined_date_polygons[date_name] = data

print(f"Combined coverage data for {len(combined_date_polygons)} dates.")

# Compute average centroid to center map
all_coords = []
for v in combined_date_polygons.values():
    for poly in v['polygons']:
        all_coords.extend(list(poly.exterior.coords))

if all_coords:
    avg_lon = sum(c[0] for c in all_coords) / len(all_coords)
    avg_lat = sum(c[1] for c in all_coords) / len(all_coords)
else:
    avg_lon, avg_lat = 0, 0

m = folium.Map(location=[avg_lat, avg_lon], zoom_start=7)

feature_groups = {}
for date_name, data in combined_date_polygons.items():
    fg = folium.FeatureGroup(name=date_name, show=False)
    for poly, meta in zip(data['polygons'], data['metas']):
        tooltip_text = (
            f"{meta['name']}<br>"
            f"Begin: {meta['begin']}<br>"
            f"End: {meta['end']}<br>"
            f"Satellite: {meta.get('satellite', 'Unknown')}"
        )
        folium.GeoJson(poly.__geo_interface__, tooltip=tooltip_text).add_to(fg)
    fg.add_to(m)
    feature_groups[date_name] = fg

folium.LayerControl(collapsed=False).add_to(m)

date_names = list(combined_date_polygons.keys())

dropdown_html = f"""
<select id="dateSelect" onchange="changeDate()" style="font-size:14px; padding:3px;">
  <option value="" selected>-- Select Date --</option>
  {''.join(f'<option value="{d}">{d}</option>' for d in date_names)}
</select>

<script>
var map = window.map || null;  // Folium map instance
var featureGroups = {json.dumps(date_names)};

// Store references to feature groups by date name
var fgLayers = {{}};

function setFeatureGroupRefs() {{
    featureGroups.forEach(function(date) {{
        var foundLayer = null;
        map.eachLayer(function(layer) {{
            if(layer.options && layer.options.pane === 'overlayPane') {{
                if(layer._popup && layer._popup._content && layer._popup._content.includes(date)) {{
                    foundLayer = layer;
                }}
            }}
        }});
        fgLayers[date] = foundLayer;
    }});
}}

function changeDate() {{
    var selected = document.getElementById('dateSelect').value;
    featureGroups.forEach(function(date) {{
        if(fgLayers[date]) {{
            if(date === selected) {{
                if(!map.hasLayer(fgLayers[date])) {{
                    map.addLayer(fgLayers[date]);
                }}
            }} else {{
                if(map.hasLayer(fgLayers[date])) {{
                    map.removeLayer(fgLayers[date]);
                }}
            }}
        }}
    }});
}}

map.whenReady(function() {{
    setFeatureGroupRefs();
}});
</script>
"""

template = Template(f"""
<div style="position: fixed; top: 10px; left: 50px; z-index: 9999; background: white; padding: 10px; border: 2px solid grey;">
  {dropdown_html}
</div>
""")

macro = MacroElement()
macro._template = template
m.get_root().add_child(macro)

output_file = 'combined_s1_coverage_map.html'
m


Parsing SatelliteData//S1C_MP_USER_20250718T173834_20250807T194000.kml ...
Combined coverage data for 21 dates.


In [42]:
from lxml import etree
from shapely.geometry import Polygon
import folium
import json
from branca.element import Template, MacroElement

ns = {'kml': 'http://www.opengis.net/kml/2.2'}

# --- Sentinel-1 parser ---

def extract_date_folders(document):
    date_folders = []
    for elem in document:
        if etree.QName(elem.tag).localname == 'Folder':
            name_el = elem.find('kml:name', ns)
            name = name_el.text if name_el is not None else ''
            # Heuristic: date folders have a nested Folder child
            if len(elem) == 2 and elem[1].tag.endswith('Folder'):
                date_folders.append((name, elem))
    return date_folders

def extract_polygons_from_folder(folder):
    polygons = []
    metas = []
    for placemark in folder:
        if etree.QName(placemark.tag).localname != 'Placemark':
            continue
        
        name_el = placemark.find('kml:name', ns)
        name = name_el.text if name_el is not None else 'NoName'
        
        timespan = placemark.find('kml:TimeSpan', ns)
        begin = timespan.find('kml:begin', ns).text if timespan is not None else None
        end = timespan.find('kml:end', ns).text if timespan is not None else None
        
        linear_ring = placemark.find('kml:LinearRing', ns)
        if linear_ring is None:
            continue
        
        coords_text = linear_ring.find('kml:coordinates', ns).text.strip()
        coords_list = []
        for c in coords_text.split():
            lon, lat, _ = c.split(',')
            coords_list.append((float(lon), float(lat)))
        
        poly = Polygon(coords_list)
        polygons.append(poly)
        metas.append({'name': name, 'begin': begin, 'end': end})
    return polygons, metas

def parse_s1_kml_file(kml_file, satellite_name):
    with open(kml_file, 'rb') as f:
        tree = etree.parse(f)
    root = tree.getroot()
    document = root.find('kml:Document', ns)
    date_folders = extract_date_folders(document)

    date_polygons = {}
    for date_name, folder in date_folders:
        nested_folder = None
        for child in folder:
            if etree.QName(child.tag).localname == 'Folder':
                nested_folder = child
                break
        if nested_folder is None:
            continue
        
        polygons, metas = extract_polygons_from_folder(nested_folder)
        for meta in metas:
            meta['satellite'] = satellite_name
        
        if date_name in date_polygons:
            date_polygons[date_name]['polygons'].extend(polygons)
            date_polygons[date_name]['metas'].extend(metas)
        else:
            date_polygons[date_name] = {'polygons': polygons, 'metas': metas}
    return date_polygons

# --- Sentinel-2 parser ---

def extract_s2_polygons_per_date(document, ns):
    date_polygons = {}

    def recurse_features(element):
        for child in element:
            tag_local = etree.QName(child.tag).localname
            if tag_local == 'Placemark':
                polygon = child.find('kml:Polygon', ns)
                if polygon is not None:
                    linear_ring = polygon.find('kml:outerBoundaryIs/kml:LinearRing', ns)
                    if linear_ring is not None:
                        coords_text = linear_ring.find('kml:coordinates', ns).text.strip()
                        coords_list = []
                        for c in coords_text.split():
                            lon, lat, *_ = c.split(',')
                            coords_list.append((float(lon), float(lat)))
                        poly = Polygon(coords_list)

                        name_el = child.find('kml:name', ns)
                        name = name_el.text if name_el is not None else 'NoName'

                        timespan = child.find('kml:TimeSpan', ns)
                        begin = timespan.find('kml:begin', ns).text if timespan is not None else None
                        end = timespan.find('kml:end', ns).text if timespan is not None else None

                        if begin:
                            date_key = begin[:10]
                        else:
                            date_key = 'Unknown'

                        if date_key not in date_polygons:
                            date_polygons[date_key] = {'polygons': [], 'metas': []}
                        date_polygons[date_key]['polygons'].append(poly)
                        date_polygons[date_key]['metas'].append({'name': name, 'begin': begin, 'end': end, 'satellite': 'S2'})
            elif tag_local in ['Folder', 'Document']:
                recurse_features(child)

    recurse_features(document)
    return date_polygons

def parse_s2_kml_file(kml_file, satellite_name):
    with open(kml_file, 'rb') as f:
        tree = etree.parse(f)
    root = tree.getroot()
    document = root.find('kml:Document', ns)
    date_polygons = extract_s2_polygons_per_date(document, ns)

    # Add satellite info to metadata
    for date_key in date_polygons:
        for meta in date_polygons[date_key]['metas']:
            meta['satellite'] = satellite_name

    return date_polygons

# --- List of files with satellite names ---

s1_files = [
    ('SatelliteData//S1A_MP_USER_20250717T171636_20250806T194000.kml', 'S1A'),
    ('SatelliteData//S1C_MP_USER_20250718T173834_20250807T194000.kml', 'S1C'),
]

s2_files = [
    ('SatelliteData//S2A_MP_ACQ__KML_20250710T150000_20250728T180000.kml', 'S2A'),
    ('SatelliteData//S2B_MP_ACQ__KML_20250717T120000_20250804T150000.kml', 'S2B'),
    ('SatelliteData//S2C_MP_ACQ__KML_20250710T120000_20250728T150000.kml', 'S2C'),
]

# --- Parse all files and merge data ---

combined_date_polygons = {}

print("Parsing Sentinel-1 files...")
for kml_file, sat_name in s1_files:
    print(f"Parsing {sat_name} file: {kml_file}")
    dp = parse_s1_kml_file(kml_file, sat_name)
    for date_name, data in dp.items():
        if date_name in combined_date_polygons:
            combined_date_polygons[date_name]['polygons'].extend(data['polygons'])
            combined_date_polygons[date_name]['metas'].extend(data['metas'])
        else:
            combined_date_polygons[date_name] = data

print("Parsing Sentinel-2 files...")
for kml_file, sat_name in s2_files:
    print(f"Parsing {sat_name} file: {kml_file}")
    dp = parse_s2_kml_file(kml_file, sat_name)
    for date_name, data in dp.items():
        if date_name in combined_date_polygons:
            combined_date_polygons[date_name]['polygons'].extend(data['polygons'])
            combined_date_polygons[date_name]['metas'].extend(data['metas'])
        else:
            combined_date_polygons[date_name] = data

print(f"Combined coverage data for {len(combined_date_polygons)} dates.")

# --- Create Folium map ---

all_coords = []
for v in combined_date_polygons.values():
    for poly in v['polygons']:
        all_coords.extend(list(poly.exterior.coords))

if all_coords:
    avg_lon = sum(c[0] for c in all_coords) / len(all_coords)
    avg_lat = sum(c[1] for c in all_coords) / len(all_coords)
else:
    avg_lon, avg_lat = 0, 0

m = folium.Map(location=[avg_lat, avg_lon], zoom_start=7)

feature_groups = {}
for date_name, data in combined_date_polygons.items():
    fg = folium.FeatureGroup(name=date_name, show=False)
    for poly, meta in zip(data['polygons'], data['metas']):
        tooltip_text = (
            f"{meta['name']}<br>"
            f"Begin: {meta['begin']}<br>"
            f"End: {meta['end']}<br>"
            f"Satellite: {meta.get('satellite', 'Unknown')}"
        )
        folium.GeoJson(poly.__geo_interface__, tooltip=tooltip_text).add_to(fg)
    fg.add_to(m)
    feature_groups[date_name] = fg

folium.LayerControl(collapsed=False).add_to(m)

date_names = list(combined_date_polygons.keys())

dropdown_html = f"""
<select id="dateSelect" onchange="changeDate()" style="font-size:14px; padding:3px;">
  <option value="" selected>-- Select Date --</option>
  {''.join(f'<option value="{d}">{d}</option>' for d in sorted(date_names))}
</select>

<script>
var map = window.map || null;
var featureGroups = {json.dumps(date_names)};
var fgLayers = {{}};

function setFeatureGroupRefs() {{
    featureGroups.forEach(function(date) {{
        var foundLayer = null;
        map.eachLayer(function(layer) {{
            if(layer.options && layer.options.pane === 'overlayPane') {{
                if(layer._popup && layer._popup._content && layer._popup._content.includes(date)) {{
                    foundLayer = layer;
                }}
            }}
        }});
        fgLayers[date] = foundLayer;
    }});
}}

function changeDate() {{
    var selected = document.getElementById('dateSelect').value;
    featureGroups.forEach(function(date) {{
        if(fgLayers[date]) {{
            if(date === selected) {{
                if(!map.hasLayer(fgLayers[date])) {{
                    map.addLayer(fgLayers[date]);
                }}
            }} else {{
                if(map.hasLayer(fgLayers[date])) {{
                    map.removeLayer(fgLayers[date]);
                }}
            }}
        }}
    }});
}}

map.whenReady(function() {{
    setFeatureGroupRefs();
}});
</script>
"""

template = Template(f"""
<div style="position: fixed; top: 10px; left: 50px; z-index: 9999; background: white; padding: 10px; border: 2px solid grey;">
  {dropdown_html}
</div>
""")

macro = MacroElement()
macro._template = template
m.get_root().add_child(macro)

output_file = 'combined_S1_S2_coverage_map.html'
m.save(output_file)
print(f"Map with combined S1 and S2 coverage saved as {output_file}")


Parsing Sentinel-1 files...
Parsing S1A file: SatelliteData//S1A_MP_USER_20250717T171636_20250806T194000.kml
Parsing S1C file: SatelliteData//S1C_MP_USER_20250718T173834_20250807T194000.kml
Parsing Sentinel-2 files...
Parsing S2A file: SatelliteData//S2A_MP_ACQ__KML_20250710T150000_20250728T180000.kml
Parsing S2B file: SatelliteData//S2B_MP_ACQ__KML_20250717T120000_20250804T150000.kml
Parsing S2C file: SatelliteData//S2C_MP_ACQ__KML_20250710T120000_20250728T150000.kml
Combined coverage data for 29 dates.
Map with combined S1 and S2 coverage saved as combined_S1_S2_coverage_map.html


In [48]:
import json
from lxml import etree
from shapely.geometry import Polygon, mapping
from datetime import datetime, timedelta

ns = {'kml': 'http://www.opengis.net/kml/2.2'}

def extract_date_folders(document):
    date_folders = []
    for elem in document:
        if etree.QName(elem.tag).localname == 'Folder':
            name_el = elem.find('kml:name', ns)
            name = name_el.text if name_el is not None else ''
            if len(elem) == 2 and elem[1].tag.endswith('Folder'):
                date_folders.append((name, elem))
    return date_folders

def extract_polygons_from_folder(folder):
    polygons = []
    metas = []
    for placemark in folder:
        if etree.QName(placemark.tag).localname != 'Placemark':
            continue

        name_el = placemark.find('kml:name', ns)
        name = name_el.text if name_el is not None else 'NoName'

        timespan = placemark.find('kml:TimeSpan', ns)
        begin = timespan.find('kml:begin', ns).text if timespan is not None else None
        end = timespan.find('kml:end', ns).text if timespan is not None else None

        linear_ring = placemark.find('kml:LinearRing', ns)
        if linear_ring is None:
            continue

        coords_text = linear_ring.find('kml:coordinates', ns).text.strip()
        coords_list = []
        for c in coords_text.split():
            lon, lat, _ = c.split(',')
            coords_list.append((float(lon), float(lat)))

        poly = Polygon(coords_list)
        polygons.append(poly)
        metas.append({'name': name, 'begin': begin, 'end': end})
    return polygons, metas

def parse_s1_kml_file(kml_file):
    with open(kml_file, 'rb') as f:
        tree = etree.parse(f)
    root = tree.getroot()
    document = root.find('kml:Document', ns)
    date_folders = extract_date_folders(document)

    date_polygons = {}
    for date_name, folder in date_folders:
        nested_folder = None
        for child in folder:
            if etree.QName(child.tag).localname == 'Folder':
                nested_folder = child
                break
        if nested_folder is None:
            continue

        polygons, metas = extract_polygons_from_folder(nested_folder)
        date_polygons[date_name] = {'polygons': polygons, 'metas': metas}
    return date_polygons

# --- User input: KML file path ---
kml_file = 'SatelliteData\S1C_MP_USER_20250718T173834_20250807T194000.kml'

# Step 1: Parse KML and extract polygons by date
date_polygons = parse_s1_kml_file(kml_file)
print(f"Extracted coverage polygons for {len(date_polygons)} dates.")

# Step 2: Define your 12 reference dates explicitly (as strings)
# Here example from 2025-07-18 to 2025-07-30 inclusive (13 days total),
# adjust exactly 12 dates if needed
start_ref = datetime.strptime("2025-07-23", "%Y-%m-%d")
reference_dates = [(start_ref + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(12)]
print("Reference dates to keep:")
print(reference_dates)

# Step 3: Filter date_polygons to keep only polygons from reference dates
filtered_polygons = []
for date_key in reference_dates:
    if date_key in date_polygons:
        polys = date_polygons[date_key]['polygons']
        metas = date_polygons[date_key]['metas']
        for poly, meta in zip(polys, metas):
            filtered_polygons.append({
                "type": "Feature",
                "geometry": mapping(poly),
                "properties": {
                    "acquisition_date": date_key,
                    "name": meta.get('name', ''),
                    "begin": meta.get('begin', ''),
                    "end": meta.get('end', '')
                }
            })
    else:
        print(f"Warning: No polygons found for reference date {date_key}")

# Step 4: Save filtered polygons as GeoJSON
geojson_output = {
    "type": "FeatureCollection",
    "features": filtered_polygons
}

output_filename = "S1C_12day_reference_coverage_plan.geojson"
with open(output_filename, "w") as f:
    json.dump(geojson_output, f, indent=2)

print(f"Saved coverage plan GeoJSON with {len(filtered_polygons)} polygons as '{output_filename}'")


Extracted coverage polygons for 21 dates.
Reference dates to keep:
['2025-07-23', '2025-07-24', '2025-07-25', '2025-07-26', '2025-07-27', '2025-07-28', '2025-07-29', '2025-07-30', '2025-07-31', '2025-08-01', '2025-08-02', '2025-08-03']
Saved coverage plan GeoJSON with 872 polygons as 'S1C_12day_reference_coverage_plan.geojson'


In [56]:
import json
from lxml import etree
from shapely.geometry import Polygon, mapping
from datetime import datetime, timedelta

ns = {'kml': 'http://www.opengis.net/kml/2.2'}

def extract_s2_polygons_per_date(document, ns):
    date_polygons = {}

    def recurse_features(element):
        for child in element:
            tag_local = etree.QName(child.tag).localname
            if tag_local == 'Placemark':
                polygon = child.find('kml:Polygon', ns)
                if polygon is not None:
                    linear_ring = polygon.find('kml:outerBoundaryIs/kml:LinearRing', ns)
                    if linear_ring is not None:
                        coords_text = linear_ring.find('kml:coordinates', ns).text.strip()
                        coords_list = []
                        for c in coords_text.split():
                            lon, lat, *_ = c.split(',')
                            coords_list.append((float(lon), float(lat)))
                        poly = Polygon(coords_list)

                        name_el = child.find('kml:name', ns)
                        name = name_el.text if name_el is not None else 'NoName'

                        timespan = child.find('kml:TimeSpan', ns)
                        begin = timespan.find('kml:begin', ns).text if timespan is not None else None
                        end = timespan.find('kml:end', ns).text if timespan is not None else None

                        # Use begin date (YYYY-MM-DD) as key
                        if begin:
                            date_key = begin[:10]
                        else:
                            date_key = 'Unknown'

                        if date_key not in date_polygons:
                            date_polygons[date_key] = {'polygons': [], 'metas': []}
                        date_polygons[date_key]['polygons'].append(poly)
                        date_polygons[date_key]['metas'].append({'name': name, 'begin': begin, 'end': end})
            elif tag_local in ['Folder', 'Document']:
                recurse_features(child)

    recurse_features(document)
    return date_polygons

# --- User input: Sentinel-2 KML file ---
s2_kml_file = r'SatelliteData\S2C_MP_ACQ__KML_20250710T120000_20250728T150000.kml'

# Parse KML file
with open(s2_kml_file, 'rb') as f:
    tree = etree.parse(f)

root = tree.getroot()
document = root.find('kml:Document', ns)

date_polygons = extract_s2_polygons_per_date(document, ns)
print(f"Extracted coverage polygons for {len(date_polygons)} dates.")

# Convert date strings to datetime and sort
date_fmt = "%Y-%m-%d"
dates_dt = []
for d in date_polygons.keys():
    try:
        dates_dt.append(datetime.strptime(d, date_fmt))
    except Exception:
        pass
dates_dt.sort()

# Define 10 reference dates starting from earliest date, spaced 10 days apart
start_ref = dates_dt[0]
reference_dates = [(start_ref + timedelta(days=i+1)).strftime(date_fmt) for i in range(10)]
print("Reference dates to keep:")
print(reference_dates)

# Filter polygons to keep only those on reference dates
filtered_polygons = []
for date_key in reference_dates:
    if date_key in date_polygons:
        polys = date_polygons[date_key]['polygons']
        metas = date_polygons[date_key]['metas']
        for poly, meta in zip(polys, metas):
            filtered_polygons.append({
                "type": "Feature",
                "geometry": mapping(poly),
                "properties": {
                    "acquisition_date": date_key,
                    "name": meta.get('name', ''),
                    "begin": meta.get('begin', ''),
                    "end": meta.get('end', '')
                }
            })
    else:
        print(f"Warning: No polygons found for reference date {date_key}")

# Save filtered polygons as GeoJSON
geojson_output = {
    "type": "FeatureCollection",
    "features": filtered_polygons
}

output_filename = "S2C_10day_reference_coverage_plan.geojson"
with open(output_filename, "w") as f:
    json.dump(geojson_output, f, indent=2)

print(f"Saved Sentinel-2 coverage plan GeoJSON with {len(filtered_polygons)} polygons as '{output_filename}'")


Extracted coverage polygons for 19 dates.
Reference dates to keep:
['2025-07-11', '2025-07-12', '2025-07-13', '2025-07-14', '2025-07-15', '2025-07-16', '2025-07-17', '2025-07-18', '2025-07-19', '2025-07-20']
Saved Sentinel-2 coverage plan GeoJSON with 429 polygons as 'S2C_10day_reference_coverage_plan.geojson'


In [57]:
import folium
import json
from shapely.geometry import shape

# List of GeoJSON files and their satellite labels
geojson_files = [
    ('S1A_12day_reference_coverage_plan.geojson', 'Sentinel-1A'),
    ('S1C_12day_reference_coverage_plan.geojson', 'Sentinel-1C'),
    ('S2A_10day_reference_coverage_plan.geojson', 'Sentinel-2A'),
    ('S2B_10day_reference_coverage_plan.geojson', 'Sentinel-2B'),
    ('S2C_10day_reference_coverage_plan.geojson', 'Sentinel-2C'),
]

all_coords = []

# First, gather all coordinates to compute map center
for filename, sat_label in geojson_files:
    with open(filename) as f:
        gj = json.load(f)
        for feature in gj['features']:
            geom = shape(feature['geometry'])
            if geom.geom_type == 'Polygon':
                all_coords.extend(list(geom.exterior.coords))
            elif geom.geom_type == 'MultiPolygon':
                for poly in geom.geoms:
                    all_coords.extend(list(poly.exterior.coords))

if all_coords:
    avg_lon = sum(c[0] for c in all_coords) / len(all_coords)
    avg_lat = sum(c[1] for c in all_coords) / len(all_coords)
else:
    avg_lon, avg_lat = 0, 0

m = folium.Map(location=[avg_lat, avg_lon], zoom_start=6)

# Add each GeoJSON as a separate layer
for filename, sat_label in geojson_files:
    with open(filename) as f:
        gj = json.load(f)
    gj_layer = folium.GeoJson(
        gj,
        name=sat_label,
        tooltip=folium.GeoJsonTooltip(fields=['acquisition_date', 'name'], aliases=['Date', 'Name']),
        style_function=lambda feature: {
            'fillColor': '#3388ff',
            'color': '#3388ff',
            'weight': 2,
            'fillOpacity': 0.3,
        }
    )
    gj_layer.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

output_file = 'sentinel_combined_coverage_map.html'
m.save(output_file)
print(f"Coverage map saved as {output_file}")


Coverage map saved as sentinel_combined_coverage_map.html


In [83]:
import folium
import json
from shapely.geometry import shape
from branca.element import Html

geojson_files = [
    ('S1A_12day_reference_coverage_plan.geojson', 'Sentinel-1A'),
    ('S1C_12day_reference_coverage_plan.geojson', 'Sentinel-1C'),
    ('S2A_10day_reference_coverage_plan.geojson', 'Sentinel-2A'),
    ('S2B_10day_reference_coverage_plan.geojson', 'Sentinel-2B'),
    ('S2C_10day_reference_coverage_plan.geojson', 'Sentinel-2C'),
]

all_coords = []
layers_data = []
m = None

for file, label in geojson_files:
    with open(file) as f:
        gj = json.load(f)

    for feature in gj["features"]:
        feature.setdefault("properties", {})
        feature["properties"]["satellite"] = label

    for feature in gj["features"]:
        geom = shape(feature["geometry"])
        if geom.geom_type == "Polygon":
            all_coords.extend(list(geom.exterior.coords))
        elif geom.geom_type == "MultiPolygon":
            for poly in geom.geoms:
                all_coords.extend(list(poly.exterior.coords))

    layer = folium.GeoJson(
        gj,
        name=label,
        tooltip=folium.GeoJsonTooltip(
            fields=["acquisition_date", "name", "satellite"],
            aliases=["Date", "Name", "Satellite"],
        ),
        style_function=lambda f: {
            "fillColor": "blue",
            "color": "blue",
            "weight": 2,
            "fillOpacity": 0.3,
        },
    )

    if m is None and all_coords:
        avg_lon = sum(c[0] for c in all_coords) / len(all_coords)
        avg_lat = sum(c[1] for c in all_coords) / len(all_coords)
        m = folium.Map(location=[avg_lat, avg_lon], zoom_start=6)

    layer.add_to(m)
    layers_data.append((layer, gj["features"], label))

all_dates = set()
for _, features, _ in layers_data:
    for f in features:
        dt = f["properties"].get("acquisition_date")
        if dt:
            all_dates.add(dt)
all_dates = sorted(all_dates)
min_date = all_dates[0] if all_dates else ""
max_date = all_dates[-1] if all_dates else ""

js_layers_data_entries = []
for _, (layer, features, sat) in enumerate(layers_data):
    leaflet_id = layer._id
    features_props = [{"acquisition_date": f["properties"]["acquisition_date"]} for f in features]
    js_entry = f"""
      {{
        layer: window.map._layers[{leaflet_id}],
        features: {json.dumps(features_props)},
        satellite: "{sat}"
      }}
    """
    js_layers_data_entries.append(js_entry)

layers_data_js = f"layersData = [{','.join(js_layers_data_entries)}];"

date_picker_html = f"""
<div style="position: fixed; top: 10px; left: 10px; z-index: 10000; background: white; padding: 10px; border: 1px solid #ccc; font-family: Arial, sans-serif; min-width: 320px;">
  <label><b>Filter Coverage by Date Range:</b></label><br/>
  <input id="startDate" type="datetime-local" value="{min_date}T00:00" style="width: 140px;" />
  <input id="endDate" type="datetime-local" value="{max_date}T23:59" style="width: 140px; margin-left: 10px;" />
  <button id="applyFilter" style="margin-left: 10px; padding: 3px 10px;">Apply</button>
</div>

<script>
document.addEventListener("DOMContentLoaded", function() {{
  var mapInstance = null;
  for (var key in window) {{
    if (key.startsWith('map_') && window[key]._leaflet) {{
      mapInstance = window[key];
      break;
    }}
  }}
  if (!mapInstance) {{
    console.error('Map instance not found');
    return;
  }}
  window.map = mapInstance;

  var layersData = [];

  function safeParseDate(dateStr) {{
    if (!dateStr) return null;
    if (dateStr.indexOf('T') === -1 && dateStr.indexOf(' ') !== -1) {{
      dateStr = dateStr.replace(' ', 'T');
    }}
    var d = new Date(dateStr);
    return isNaN(d) ? null : d;
  }}

  {layers_data_js}

  function removeAllLayers() {{
    layersData.forEach(function(ld) {{
      if (window.map.hasLayer(ld.layer)) {{
        window.map.removeLayer(ld.layer);
      }}
    }});
  }}

  function addLayer(ld) {{
    if (!window.map.hasLayer(ld.layer)) {{
      window.map.addLayer(ld.layer);
    }}
  }}

  function filterLayers(startDate, endDate) {{
    var visibleSats = new Set();
    var layersControl = window.map._controlLayers;
    if (layersControl) {{
      var inputs = layersControl._form.getElementsByTagName('input');
      for(var i=0; i<inputs.length; i++) {{
        var input = inputs[i];
        if (input.checked) {{
          visibleSats.add(input.nextSibling.textContent.trim());
        }}
      }}
    }}

    removeAllLayers();

    layersData.forEach(function(ld) {{
      var sat = ld.satellite;
      var show = false;
      if(visibleSats.has(sat)) {{
        for(var i=0; i<ld.features.length; i++) {{
          var f = ld.features[i];
          var fDate = safeParseDate(f.acquisition_date);
          if(fDate && fDate >= startDate && fDate <= endDate) {{
            show = true;
            break;
          }}
        }}
      }}
      if(show) {{
        addLayer(ld);
      }}
    }});
  }}

  document.getElementById('applyFilter').addEventListener('click', function() {{
    var startInput = document.getElementById('startDate').value;
    var endInput = document.getElementById('endDate').value;
    if(!startInput || !endInput) {{
      alert('Please set both start and end datetime values.');
      return;
    }}
    var startDate = new Date(startInput);
    var endDate = new Date(endInput);
    filterLayers(startDate, endDate);
  }});

  function initializeLayersData() {{
    layersData = [];
  }}

  initializeLayersData();
}});
</script>
"""

# Use branca.element.Html instead of Element to avoid Jinja parsing issues
from branca.element import Html
m.get_root().html.add_child(Html(date_picker_html, script=True))

folium.LayerControl().add_to(m)

output_file = "sentinel_map_with_layer_addremove_filter.html"
m.save(output_file)
print(f"Map saved as {output_file}. Open it in a browser and test filtering.")

Map saved as sentinel_map_with_layer_addremove_filter.html. Open it in a browser and test filtering.


In [ ]:
import folium
import geopandas as gpd
import pandas as pd
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import os

# List of GeoJSON files (replace with your file paths)
satellite_files = ['S1A_12day_reference_coverage_plan.geojson', 'S1C_12day_reference_coverage_plan.geojson']  # Adjust as needed

# Load all GeoJSON files and combine into a single GeoDataFrame
def load_satellite_data(files):
    all_gdfs = []
    for file in files:
        if os.path.exists(file):
            gdf = gpd.read_file(file)
            # Ensure acquisition_date is in datetime format
            gdf['acquisition_date'] = pd.to_datetime(gdf['acquisition_date'])
            # Add satellite identifier
            gdf['satellite'] = os.path.splitext(os.path.basename(file))[0]  # e.g., 'satellite1'
            all_gdfs.append(gdf)
        else:
            print(f"File {file} not found.")
    if not all_gdfs:
        raise ValueError("No valid GeoJSON files loaded.")
    return gpd.GeoDataFrame(pd.concat(all_gdfs, ignore_index=True))

# Load data
try:
    data = load_satellite_data(satellite_files)
except ValueError as e:
    print(e)
    exit()

# Create date picker widget
date_picker = widgets.DatePicker(
    description='Select Date',
    value=data['acquisition_date'].min().date(),  # Default to earliest date
    min=data['acquisition_date'].min().date(),
    max=data['acquisition_date'].max().date()
)

# Define colors for different satellites
colors = ['blue', 'red', 'green', 'purple', 'orange']  # Add more if needed
satellite_colors = {sat: colors[i % len(colors)] for i, sat in enumerate(data['satellite'].unique())}

# Function to update map based on selected date
def update_map(change):
    clear_output(wait=True)  # Clear previous output
    display(date_picker)  # Redisplay date picker
    
    # Get selected date
    selected_date = change['new']
    if selected_date is None:
        print("Please select a date.")
        return
    
    # Filter data for the selected date
    selected_date = pd.to_datetime(selected_date)
    filtered_data = data[data['acquisition_date'].dt.date == selected_date.date()]
    
    # Create a new Folium map
    m = folium.Map(location=[0, 0], zoom_start=2)  # Centered on globe
    
    # Add polygons for each satellite
    for satellite in filtered_data['satellite'].unique():
        sat_data = filtered_data[filtered_data['satellite'] == satellite]
        for _, row in sat_data.iterrows():
            # Convert geometry to GeoJSON format for Folium
            geojson_data = json.loads(row['geometry'].to_json())
            folium.GeoJson(
                geojson_data,
                name=satellite,
                style_function=lambda x, color=satellite_colors[satellite]: {
                    'fillColor': color,
                    'color': color,
                    'weight': 2,
                    'fillOpacity': 0.5
                },
                popup=folium.Popup(
                    f"{satellite}<br>Date: {row['acquisition_date'].strftime('%Y-%m-%d')}<br>"
                    f"Time: {row['begin']} to {row['end']}"
                )
            ).add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Display the map

    m.save('satellite_coverage_map.html')  # Save map to HTML file
    display(m)

# Link date picker to update function
date_picker.observe(update_map, names='value')

# Initial display
display(date_picker)

DatePicker(value=datetime.date(2025, 7, 18), description='Select Date', max=datetime.date(2025, 8, 3), min=dat…